In [ ]:
!pip install -q transformers datasets accelerate

In [ ]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

model = GPT2LMHeadModel.from_pretrained("gpt2")
model.resize_token_embeddings(len(tokenizer))

Embedding(50257, 768)

In [ ]:
!pip install pandas

In [ ]:
import csv

texts = []

with open("train.csv", encoding="utf-8", errors="ignore") as f:
    reader = csv.DictReader(f)
    for row in reader:
        if "text" in row and row["text"]:
            texts.append(row["text"].replace("\n", " "))

with open("validation.csv", encoding="utf-8", errors="ignore") as f:
    reader = csv.DictReader(f)
    for row in reader:
        if "text" in row and row["text"]:
            texts.append(row["text"].replace("\n", " "))

print("Total stories:", len(texts))

Total stories: 247950


In [ ]:
print("Total stories extracted:", len(texts))

Total stories extracted: 247950


In [ ]:
import random

for i in random.sample(range(len(texts)), 3):
    print("\n--- STORY SAMPLE ---")
    print(texts[i][:500])


--- STORY SAMPLE ---
Lily likes to collect pink things. She has a pink dress, a pink hat, a pink doll and a pink bike. She likes to put them in her pink room and look at them. She thinks they are pretty and make her happy.  One day, Lily goes to the park with her mom. She sees a big pile of pink leaves. She runs to the pile and starts to collect them. She wants to take them home and put them in her room. She thinks they are pretty and make her happy.  But when she picks up the leaves, she smells something bad. The l

--- STORY SAMPLE ---
Mia and Leo like to pretend. They pretend to be animals, or pirates, or superheroes. They use their toys and their clothes and their imagination.  One day, they pretend to be chefs. They find some prunes in the kitchen. Prunes are dried plums. They are wrinkly and sweet. Mia and Leo think they are funny.  They put some prunes in a bowl and pretend to mix them. They pretend to add some salt and some sugar and some water. They pretend to cook them on th

In [ ]:
lengths = [len(t.split()) for t in texts]

print("Min words:", min(lengths))
print("Max words:", max(lengths))
print("Average words:", sum(lengths) // len(lengths))

Min words: 13
Max words: 963
Average words: 175


In [ ]:
texts = [t for t in texts if len(t.split()) > 20]
print("After cleaning:", len(texts))

After cleaning: 247949


In [ ]:
with open("training_data.txt", "w", encoding="utf-8") as f:
    for story in texts:
        f.write(story.strip() + "\n")

In [ ]:
!wc -l training_data.txt

247949 training_data.txt


In [ ]:
!head -n 3 training_data.txt

One day, a little girl named Lily found a needle in her room. She knew it was difficult to play with it because it was sharp. Lily wanted to share the needle with her mom, so she could sew a button on her shirt.  Lily went to her mom and said, "Mom, I found this needle. Can you share it with me and sew my shirt?" Her mom smiled and said, "Yes, Lily, we can share the needle and fix your shirt."  Together, they shared the needle and sewed the button on Lily's shirt. It was not difficult for them because they were sharing and helping each other. After they finished, Lily thanked her mom for sharing the needle and fixing her shirt. They both felt happy because they had shared and worked together.
Once upon a time, there was a little car named Beep. Beep loved to go fast and play in the sun. Beep was a healthy car because he always had good fuel. Good fuel made Beep happy and strong.  One day, Beep was driving in the park when he saw a big tree. The tree had many leaves that were falling. B

In [ ]:
!nvidia-smi

Mon Jan 26 17:34:42 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   39C    P8              9W /   70W |       2MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
from datasets import load_dataset

dataset = load_dataset(
    "text",
    data_files={"train": "training_data.txt"}
)

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
print(dataset)
print(dataset["train"][0]["text"][:300])

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 247949
    })
})
One day, a little girl named Lily found a needle in her room. She knew it was difficult to play with it because it was sharp. Lily wanted to share the needle with her mom, so she could sew a button on her shirt.  Lily went to her mom and said, "Mom, I found this needle. Can you share it with me and 


In [ ]:
def tokenize_function(examples):
    return tokenizer(
        examples["text"],
        truncation=True,
        padding="max_length",
        max_length=128
    )

tokenized_dataset = dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=["text"]
)

Map:   0%|          | 0/247949 [00:00<?, ? examples/s]

In [ ]:
from transformers import TrainingArguments

#training_args = TrainingArguments(
 #   output_dir="./gpt2-tinystories",
  #  overwrite_output_dir=True,
  #  num_train_epochs=1,
   # max_steps=5000,
   # per_device_train_batch_size=2,
   # gradient_accumulation_steps=4,
   # save_steps=5000,
   # save_total_limit=2,
   # logging_steps=1000,
   # learning_rate=5e-5,
   # fp16=True,
   # report_to="none"
#)
training_args = TrainingArguments(
    output_dir="./gpt2-tinystories",
    overwrite_output_dir=True,

    num_train_epochs=3,                # ⬅️ higher epochs
    per_device_train_batch_size=4,     # ⬅️ speed boost
    gradient_accumulation_steps=4,     # effective batch = 16

    learning_rate=5e-5,
    fp16=True,

    save_strategy="no",                # ⬅️ HUGE time saver
    logging_steps=1000,

    report_to="none"
)

In [ ]:
from transformers import DataCollatorForLanguageModeling

In [ ]:
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False  # VERY IMPORTANT for GPT-2
)

In [ ]:
from transformers import Trainer

train_subset = tokenized_dataset["train"] \
    .shuffle(seed=42) \
    .select(range(30000))

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_subset,
    data_collator=data_collator
)
# train_dataset=tokenized_dataset["train"]

In [ ]:
trainer.train()

Step,Training Loss
1000,1.625500
2000,1.594000
3000,1.538900
4000,1.519600
5000,1.493800


TrainOutput(global_step=5625, training_loss=1.5469735460069445, metrics={'train_runtime': 1740.601, 'train_samples_per_second': 51.706, 'train_steps_per_second': 3.232, 'total_flos': 5879070720000000.0, 'train_loss': 1.5469735460069445, 'epoch': 3.0})

In [ ]:
import re

def trim_to_last_sentence(text):
    matches = list(re.finditer(r'[.!?]', text))
    if matches:
        last_end = matches[-1].end()
        return text[:last_end]
    return text

In [ ]:
#prompt = "Once upon a time"
#inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

#outputs = model.generate(
 #   **inputs,
  #  max_length=1000,
   # temperature=0.8,
    #top_p=0.9,
    #repetition_penalty=1.2,
    #no_repeat_ngram_size=3,
    #eos_token_id=tokenizer.eos_token_id,
    #do_sample=True
#)

#raw_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
#final_text = trim_to_last_sentence(raw_text)

#print(final_text)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Once upon a time I thought that there was no reason to write about this. The book did not come out and it had been almost two years since the first one, so we were still waiting for another publication date of sorts when they said: "No!" So what? Well at least now you know why!
The story takes place in an ordinary university with lots more than 100 people working on our research - some doing their jobs while others are writing stories themselves as well; but sometimes just getting things done or talking back into something is necessary if your work interests don't seem very keen by today's standards (which happens). In fact many students who've started up new projects have turned down these chances because other teachers couldn�t afford them anyway – all those books do will be published once again after 20-30 pages which means most writers won`nt bother paying attention anymore.' A huge amount can go wrong here though...

 'A lot of young children want kids playing games' : ''It seems 

In [ ]:
#for i in range(3):
 #   print(f"\n--- STORY {i+1} ---")
  #  outputs = model.generate(
   #     **inputs,
    #    max_length=1000,
     #   temperature=0.75,
      #  top_p=0.9,
       # repetition_penalty=1.2,
       # no_repeat_ngram_size=3,
        #eos_token_id=tokenizer.eos_token_id,
        #do_sample=True
    #)
    #text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    #print(trim_to_last_sentence(text))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



--- STORY 1 ---


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Once upon a time the whole of his life had been lost. He was now in deep grief for what he saw, but no one could give him up to it; so that though there were people who said they loved me and I would die when my face is gone from them (which seems too much) or if God might have told us all about this matter at some point we'd be better off without Him than with any other man's suffering."
When asked why Jesus wanted others sufferings "in order not only as an act of love," which many say has nothing whatsoever to do—but even more because most Christian theologians deny Christ exists–he replied: "[T]hey can't possibly exist apart by chance between two persons whom neither know nor care how old their eyes are!" In fact these days those familiar enough with religious history will remember quite well exactly where once such things began again after Paul took on flesh-and-"womb" captivity around His last supper . But while each person must take responsibility here then – whether through fear

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Once upon a time, the two were going to die. But as I listened in awe at how close their lives seemed together and we knew they'd both be gone by now it was clear that my prayers would have been answered if not for all those lost comrades who had sacrificed so much more than anything else on earth before me."
The woman smiled sadly; she did seem quite happy about her situation because of what happened but there's no way around this fact: Her daughter died too young after he became an adult – which means his father may never know where or when… It makes you wonder why people with such limited experience can believe any kind person could possibly kill someone like these kids? My response is simple yet very important here since even though some might think otherwise many are simply using words just wrongfully against anyone involved (see below).

[…] Asking 'what do your ancestors really feel' goes without saying unless something similar happens today... Why does anybody care whether our 

In [ ]:
base_model = GPT2LMHeadModel.from_pretrained("gpt2").to("cuda")
base_tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

inputs = base_tokenizer(prompt, return_tensors="pt").to("cuda")

outputs = base_model.generate(
    **inputs,
    max_new_tokens=500,           # not 1000
    temperature=0.7,              # lower randomness
    top_p=0.9,
    repetition_penalty=1.3,       # 🔑 prevents looping
    no_repeat_ngram_size=3,       # 🔑 blocks phrase repetition
    eos_token_id=base_tokenizer.eos_token_id,
    pad_token_id=base_tokenizer.eos_token_id,
    do_sample=True
)


print(base_tokenizer.decode(outputs[0], skip_special_tokens=True))

Once upon a time, a child discovered something magical in the forest.
The magic was there for all to see and hear when he'd seen it before... but now? It's only been one year since his first visit back home… no more! Now that I think about this new revelation from my daughter she is really starting to realize how much of her life has changed as well!" (Rei)

- Rei: "Yes…" The children are finally awake enough to tell their parents after they've left school.. And then everyone comes out into town with them.... so why don't you go on your own adventures!? Just ask me what happened here…. or would anyone find anything interesting?!" ...I'm going to try not get too caught up trying things like asking other people who were at least once friends if any could be brought along….. Maybe even just look through some pictures!! :P


In [ ]:
#with open("generated_samples.txt", "w", encoding="utf-8") as f:
 #   for i in range(5):

        # 🔑 Ensure inputs are on the SAME device as the model
  #      inputs = {k: v.to(model.device) for k, v in inputs.items()}

   #     outputs = model.generate(
    #        **inputs,
     #       max_new_tokens=1000,          # use this instead of max_length
      #      temperature=0.75,
       #     top_p=0.9,
        #    repetition_penalty=1.2,
         #   no_repeat_ngram_size=3,
          #  eos_token_id=tokenizer.eos_token_id,
           # pad_token_id=tokenizer.eos_token_id,
            #do_sample=True
        #)

        #raw_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
        #final_text = trim_to_last_sentence(raw_text)

        # ✅ PRINT output
        #print(f"\n--- GENERATED STORY {i+1} ---\n")
        #print(final_text)

        # ✅ SAVE output
        #f.write(final_text + "\n\n")


--- GENERATED STORY 1 ---

Once upon a time, the United States had been accused of having violated its founding principles and traditions. The first U .S.-backed invasion was launched in August 1941 after World War II ended with Pearl Harbor being struck by Japanese aircraft as they tried to attack Tokyo's southern coast — which later became part Asian territory for Japan under Allied occupation during WWII.

--- GENERATED STORY 2 ---

Once upon a time the sky was blue and clear. There were no clouds nor fog, just an empty air; everything seemed to be centered on one spot or another…

"Hmph!" The light of wind blew from afar towards him in all directions as if he had been falling down onto it like that! It suddenly felt so good!! "What do you think?" He looked at Lin Feng with worry over his eyesight but finally understood something: not only did this man have dark hair (He couldn't remember what colour), which caused other people's bodies within them tremble slightly after looking up

In [ ]:
demo_prompts = [
    "Once upon a time, there was a kind little girl who lived in a small village.",
    "Once upon a time, a small animal learned an important lesson about friendship.",
    "Once upon a time, a child discovered something magical in the forest."
]

In [ ]:
generation_kwargs = dict(
    max_new_tokens=200,            # enough for full story
    temperature=0.55,              # reduces drift
    top_p=0.9,
    repetition_penalty=1.2,
    no_repeat_ngram_size=3,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id,
    do_sample=True
)

In [ ]:
with open("generated_samples.txt", "w", encoding="utf-8") as f:
    for i, prompt in enumerate(demo_prompts, 1):

        inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

        outputs = model.generate(
            **inputs,
            **generation_kwargs
        )

        raw_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
        final_text = trim_to_last_sentence(raw_text)

        print(f"\n--- DEMO STORY {i} ---\n")
        print(final_text)

        f.write(f"--- DEMO STORY {i} ---\n")
        f.write(final_text + "\n\n")


--- DEMO STORY 1 ---

Once upon a time, there was a kind little girl who lived in a small village. She loved to play with her friends and eat yummy food. One day she found an enormous box in the woods near where they were playing hide-and seek! The little girl was very excited and tried hiding behind it all by herself until she was sure nobody would find her.   Suddenly, a big bear appeared from outside the box and said: "I'm here to catch you!" The little boy was scared and ran away as fast he could but his mommy said no again. The bear was so angry that he started barking at the boy and shouting even louder than before - he had been hiding for hours!   The boy was taken away and his mommies were very sad. They knew the little girl's momma didn't mean to him and they learned that sometimes things can happen quickly and safely. From then on when they were reunited, the little boy felt much better knowing that he was safe and secure in his new home.

--- DEMO STORY 2 ---

Once upon a t